In [34]:
# TODO: add all necessary imports
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if gpu else "cpu")
print("device:", device)

In [ ]:
# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),  # Converts PIL image to tensor (0-1 range)
    transforms.Normalize((0.5,), (0.5,))  # Normalize to (-1, 1) range
])

# Load training data
train_dataset = datasets.MNIST(
    root='./data',           # Where to download/store data
    train=True,              # Load training set
    download=True,           # Download if not present
    transform=transform      # Apply transforms
)

# Load test data
test_dataset = datasets.MNIST(
    root='./data',
    train=False,             # Load test set
    download=True,
    transform=transform
)

# Create data loaders for batching
train_loader = DataLoader(
    train_dataset,
    batch_size=128,           # Number of samples per batch
    shuffle=True             # Shuffle training data
)

test_loader = DataLoader(
    test_dataset,
    batch_size=128,           # Number of samples per batch
    shuffle=True             # Shuffle training data
)

batch = next(iter(train_loader))
x_batch, y_batch = batch

print(x_batch.shape)
print(y_batch.shape)
print("test2")

torch.Size([128, 1, 28, 28])
torch.Size([128])


In [ ]:
class FashionMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        in_features = 784
        out_features = len(train_dataset.classes)
        self.fc1 = nn.Linear(in_features, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, out_features)  # 10 classes
        self.relu = nn.ReLU()

        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        for layer in [self.fc1, self.fc2, self.fc3]:
            nn.init.kaiming_normal_(layer.weight)
            layer.bias.data.zero_()
            
    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# initialize the model 
model = FashionMLP()
# Loss function 
loss_function = nn.CrossEntropyLoss()
# Optimizer 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

# Number of epochs 
n_epoch = 30 

for epoch in tqdm(range(n_epoch), desc="Epochs"):
    epoch_loss = 0.0
    for x_batch, y_batch in train_loader: 
        optimizer.zero_grad() 
        pred = model(x_batch)
        loss = loss_function(pred, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
# print error
print(f'Epoch {epoch:5d}, loss {epoch_loss:.3f}')
# tell scheduler to consider updating learning rate
scheduler.step()

Epochs:   0%|          | 0/30 [00:00<?, ?it/s]